In [1]:
#I run this in terminal below. for some reason it does not work here
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

/bin/bash: wget: command not found


In [2]:
with open ('./input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
from pprint import pprint
print(text[-1000:])

 while you take your rest,
And watch your safety.

ALONSO:
Thank you. Wondrous heavy.

SEBASTIAN:
What a strange drowsiness possesses them!

ANTONIO:
It is the quality o' the climate.

SEBASTIAN:
Why
Doth it not then our eyelids sink? I find not
Myself disposed to sleep.

ANTONIO:
Nor I; my spirits are nimble.
They fell together all, as by consent;
They dropp'd, as by a thunder-stroke. What might,
Worthy Sebastian? O, what might?--No more:--
And yet me thinks I see it in thy face,
What thou shouldst be: the occasion speaks thee, and
My strong imagination sees a crown
Dropping upon thy head.

SEBASTIAN:
What, art thou waking?

ANTONIO:
Do you not hear me speak?

SEBASTIAN:
I do; and surely
It is a sleepy language and thou speak'st
Out of thy sleep. What is it thou didst say?
This is a strange repose, to be asleep
With eyes wide open; standing, speaking, moving,
And yet so fast asleep.

ANTONIO:
Noble Sebastian,
Thou let'st thy fortune sleep--die, rather; wink'st
Whiles thou art waking.


In [4]:
# let look at how long the text is:
print ("length of the text is:", len(text))

length of the text is: 1115394


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print (''.join(chars))
print (vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


<h1>Tokenization</h1>

Note in the previous step we defined our vocabulary. A vcabulary is basicall a possilbe  characters that the model can see or emit. Next we will develop some strategy to tkenize the input text. Now when people ssay tokenize they mean covert raw text as a string to some sequenfe of sintegers acording to some vocabulary of possible elements. As an example here we are buidlinga character level language model we are simply going to translate individual characters into integers.

In [22]:
# Lets create mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]            #take a string and output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])   #take a list of integers and output a string

print (encode("hii there"))
print (decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


Different implementations of this are around. for example google uses the part of sentence tokenization. Sentence piece is a sub word unit level tokenization which is usually what is implemented in practice. Models like GPT uses TikToken that uses bite pair encoding (BPE) tokenizer. That is what also GPT uses. 

In [23]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab)
print( enc.encode('hii there'))
print(enc.decode([71, 4178, 612]))

50257
[71, 4178, 612]
hii there


note the interegsre. basically you can trade off code book size and sequence lengths. you can have very long sequence of integer with very small vocab. or you can have short sequences of integers with very large vocabulary. chck 12:00

In [27]:
# we have encoder and decoder. therefore we can encoxe the whol tinyshakespear
# let's now encode the entire text dataset and store it into a torch.tensor
import torch #
data = torch.tensor(encode(text), dtype=torch.long) # by default torch cast to float, but we need to make it integer therefore the dtype=torch.long
print (data.shape, data.dtype)
print (data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [28]:
# Let's now split our data into train and validation sets (this will help us understand to what degree the model is overfitting)
n = int(0.9*len(data)) # first 90% will be the training data, rest val 
train_data = data[:n]
valid_data = data[n:]

Note that we can't feed the whole test to the transformer for training. This will be computationally very expensieve and prohibitive. Therefore we feed only chunks of the dataset. Wee sample random little chunkcs out of the training set and train on these chuncks. These chunkcs have some maximum length. the maximum lngth is typically called the Block size/ Context length

In [30]:
block_size = 8
train_data[:block_size+1] # we sill see why +1

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

when you sample a chunk of data like this, ie the 9 characters out of dataset. the window is cumulative here

In [32]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print (f"when input is {context} target is {target}")

when input is tensor([18]) target is 47
when input is tensor([18, 47]) target is 56
when input is tensor([18, 47, 56]) target is 57
when input is tensor([18, 47, 56, 57]) target is 58
when input is tensor([18, 47, 56, 57, 58]) target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


so there are 8 example above. We train the transofmrer model for each of the steps above. from 1 all the way up to the block size. Transformer will be able to predict up to block size. then we have to truncate. The above is basically a time demention. then we have to look at the batch dimension. As we are sampling these chunks of text, everytime we are fedding them into a transformer we are going to have many batches of mulitple chuynks of text that are all stacked up in a single tensor. this is done for efficiency to keep the GPUs busy. because GPUs are very good with parallel processing of the data. We want to process multiple chunks all at the same time. THose chunks are odne completely inpdepndently and they dont talk to heach other.

In [39]:
torch.manual_seed(1337)
batch_size = 4 # How many indpendent sequences we are processing in parallel?
block_size = 8 #what is the maximum context length of prediction

def get_batch(split):
    # generate a small batch of data of input x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size, )) # this basically pick some random integers form the whole text and (batch_size, ) gives the number of those random integers in our case there will be 4 of them
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print ('inputs:')
print(xb.shape)
print(xb)
print ('targets:')
print (yb.shape)
print (yb)

print ('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b, t]
        print (f"when input is {context.tolist()} the target is {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44, 53, 56, 1, 58] the target 

In [36]:
torch.randint(len(data)-block_size, (batch_size, ))

tensor([1081275,  241627,  748567,  905830])

In [40]:
print (xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [42]:
# Let's tart with the most simple language model which is the bi-gram language modeling
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) 
        
    def forward(self, idx, targets):
        # idx and target are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C) batch(=4), time(=8), channel(=vocab_size=65)
        #logits are basically the score for the next character to come
        return logits 
    

m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print (out.shape)
print (out)

torch.Size([4, 8, 65])
tensor([[[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
         [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
         [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
         ...,
         [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
         [ 1.0901,  0.2170, -2.9996,  ..., -0.5472, -0.8017,  0.7761],
         [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594]],

        [[ 1.0541,  1.5018, -0.5266,  ...,  1.8574,  1.5249,  1.3035],
         [-0.1324, -0.5489,  0.1024,  ..., -0.8599, -1.6050, -0.6985],
         [-0.6722,  0.2322, -0.1632,  ...,  0.1390,  0.7560,  0.4296],
         ...,
         [ 1.0901,  0.2170, -2.9996,  ..., -0.5472, -0.8017,  0.7761],
         [ 1.1513,  1.0539,  3.4105,  ..., -0.5686,  0.9079, -0.1701],
         [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305]],

        [[-0.2103,  0.4481,  1.2381,  ...,  1.3597, -0.0821,  0.3909],
         [ 0.2475, -0.